### Thread vs coroutine

By calling thread_obj.join(), we will wait for the thread_obj to terminate before continuing execute the next code.

By default, there is no way to explicitly tell the thread to stop. We have to setup the stop-condition inside the thread itself.

asyncio uses coroutines (i.e. await). Coroutines are asynchronous by default.

With coroutines, when a function wants to give up control, it may yield, yield from or await. The control is then given back to the scheduler.
<br>
The function that call the coroutine will continue executing the next lines of code.

Code with yield from (await) is often hard to read.
<br>
Guido suggests that we can just ignore this term when reading. The same code would be executed, except that it would be blocking. (With await, it is not blocking.)

In [16]:
import asyncio
import time
import random

In [24]:
def action_single(inp : int) -> str:
    print('input {}'.format(inp))
    time.sleep(inp)
    print('return {}'.format(inp*10))
    return inp*10

def action_multi(inps : list) -> list:
    print('in action_multi')
    
    result = [action_single(inp) for inp in inps]
    print('result: {}'.format(result))
    
    return result

if __name__ == '__main__':
    inps = range(1, 5)
    action_multi(inps)

in action_multi
input 1
return 10
input 2
return 20
input 3
return 30
input 4
return 40
result: [10, 20, 30, 40]


In [23]:
async def action_single(inp : int) -> str:
    print('input {}'.format(inp))
    await asyncio.sleep(inp)
    print('return {}'.format(inp*10))
    return inp*10

async def action_multi(inps : list) -> list:
    print('in action_multi')
    
    result = await asyncio.gather(
        *[action_single(inp) for inp in inps]
    )
    print('result: {}'.format(result))
    
    return result

if __name__ == '__main__':
    inps = range(1, 5)
    await action_multi(inps) # outside Jupyter: asyncio.run(action_multi(inps))

in action_multi
input 1
input 2
input 3
input 4
return 10
return 20
return 30
return 40
result: [10, 20, 30, 40]


An event loop basically says: "when event A happens, react with function B".

Instead of using:
<br>
@asyncio.coroutine
def my_coro():
    yielf from
   
Now we can use:
<br>
async def my_coro():
    await

Note that await can only be used inside async def.

The keyword await passes function's control back.

If we see, inside function g() there is a call to await f(), this means: suspend the function g() until f() returns; in the mean time, do something else (continue executing the function that called g() ).

To call a coroutine, we must await it to get its result.

My tip for async: whenever we need to get result from a function f(), use await.

asyncio.Queue class is similar to queue.Queue, but used for async tasks rather than multi-threads.

async.Queue:
<br>
q.put()
<br>
q.get()
<br>
q.task_done() to indicate that the just-pop element was handled successfully.
<br>
q.join() blocks until all tasks in the queue are handled. To make it non-blocking: await q.join()


asyncio.create_task(coro) creates and schedules a task. This returns a Task object.

aiopg is a package that supports synchronous Postgresql that is compatible with asyncio.